In [1]:
import base64
import json

import pandas as pd
import requests
import tensorflow as tf

In [2]:
df = pd.read_csv("data/cyberbullying.csv")

df.head()


,label,text
0,0,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,0,Geblek lo tata...cowo bgt dibela2in balikan......
2,0,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,0,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,0,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."


In [3]:
test_text = df["text"][0]

test_text


" <USERNAME> TOLOL!! Gak ada hubungan nya keguguran dgn pake hijab syar'i yg lo bilang bayi nya kepanasan didalem gak ada hubungan nya woyyyy!! Otak sama jempol lo gak singkron sih ya jadinya asal nulis komentar!"

In [4]:
def prepare_json(text):
    feature_spec = {
        "text": tf.train.Feature(
            bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")])
        )
    }

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps({"signature_name": "serving_default", "instances": result})


In [5]:
def make_predictions(text):
    json_data = prepare_json(text)

    endpoint = "http://localhost:8080/v1/models/cyberbullying-detection-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]
    return prediction


In [6]:
# Prediction
make_predictions(test_text)


0.119945735

In [7]:
# Actual label
df["label"][1]

0